In [2]:
import torch
from reggae.utilities import save, load

from reggae.gp.variational.models import NonLinearLFM, ExponentialLFM
from reggae.gp.variational.trainer import Trainer
from reggae.data_loaders.artificial import get_artificial_dataset
from reggae.plot.variational_plotters import Plotter
from matplotlib import pyplot as plt

import numpy as np

f64 = np.float64

In [ ]:
replicate = 0
nodelay_dataset, delay_dataset = get_artificial_dataset()
p_nodelay, m_nodelay = nodelay_dataset

In [ ]:
print(p_nodelay.shape)
print(m_nodelay.shape)

m_nodelay = m_nodelay[replicate]
p_nodelay = p_nodelay[replicate]
num_genes = m_nodelay.shape[0]
num_tfs = p_nodelay.shape[0]

In [ ]:
model = NonLinearLFM(num_genes, num_tfs, t_inducing, t_observed, fixed_variance=None)
optimizer = torch.optim.Adam(model.parameters(), lr=0.2)
trainer = Trainer(model, optimizer, (t_observed, m_observed))

# list(model.parameters())
print(t_observed.shape, m_observed.shape)

### Outputs prior to training:

In [ ]:
plotter = Plotter(model, m_df.index, t_inducing)
plot_genes(rtol=1e-3, atol=1e-4)
plotter.plot_tfs()

In [ ]:
tol = 1e-3
# trainer = Trainer(optimizer)
output = trainer.train(40, rtol=tol, atol=tol/10, report_interval=1, plot_interval=2)

### Outputs after training

In [ ]:
plotter = Plotter(model, m_df.index, t_inducing)

tol = 1e-3
plotter.plot_losses(trainer, last_x=100)
plot_genes(rtol=tol, atol=tol/10)
plotter.plot_tfs(ylim=(-2, 9))


In [ ]:
save(model, 'nonlinear')

In [ ]:
model = load('nonlinear', NonLinearLFM, num_genes, num_tfs, t_inducing, t_observed, fixed_variance=None)
optimizer = torch.optim.Adam(model.parameters(), lr=0.2)
trainer = Trainer(model, optimizer, (t_observed, m_observed))

In [ ]:
titles = ['basal', 'decay', 'sensitivity', 'lengthscale']
datas = [np.array(trainer.basalrates)[:,:,0],
         np.array(trainer.decayrates)[:,:,0],
         np.array(trainer.sensitivities)[:,:,0],
         np.array(trainer.lengthscales)]

plt.figure(figsize=(5, 6))
for i, (title, data) in enumerate(zip(titles, datas)):
    plt.subplot(411 + i)
    plt.title(title)
    # if data.ndim > 1:
    #     for j in range(data.shape[1]):

    plt.plot(data)